# Process raw Actigraph files to calculate TEE expenditure. 
In this version, epochs with "long" gaps in the HR data are filled in BMR, and epochs with "short" gaps in the HR data are interpolated. Missing data at the start and end of the day are also filled in with BMR.

++++++++++++++++++

Brendan Croom<br>
b.p.croom@gmail.com

Created: 22 JAN 2020<br>
Last Updated: 31 AUG 2020

++++++++++++++++++

Code to "auto-magically" process raw HR data files based on the following operations:
1. Interpolate the "HR = 0" data during "short gaps" of t < 15 minutes
2. Identify and flag "HR = 0" data during long gaps (t > 15 minutes). TEE for these points will later be filled in with BMR.
3. Calculate TEE according to Hiiloskorpi formula. The TEE is normalized per minute (simply sum to calculate total TEE in a day)
4. Ensure that TEE is greater than or equal to BMR. Excessively small TEE values are replaced with BMR.
5. Data is saved in a new Excel file, with a tab corresponding to each unique day

In [ ]:
import os
import numpy as np
import pandas as pd
from scipy import interpolate, optimize
from matplotlib import pyplot as plt
import datetime

# Note... also need to have XlsxWriter installed in background. Refer
# to https://xlsxwriter.readthedocs.io/index.html

# Write the methods used to process the data

First, we define the Hiiloskorpi and Dugas formulas used to calculate the total energy expenditure (TEE) from the participants weight, heart rate and/or age. Note that the calculations depend if participant is Male vs. Female.

Additionally, the Hiiloskorpi calculation depends on the instantaneous heart rate. Specifically, if the HR > 90 beats per minute we use the "heavy activity" formula; otherwise, use the "light activity" formula.

In [ ]:
def TEE_hiiloskorpi_light(hr, weight, male=True):
    if male:
        return 4.56 - (0.0265 * hr)-(0.1506 * weight) + (0.00189 * hr * weight)
    else:
        return -4.7 + (0.0449 * hr)-(0.0019 * weight) + (0.00052 * hr * weight)
def TEE_hiiloskorpi_heavy(hr, weight, male=True):
    if male: 
        return 3.56 - (0.0138 * hr) - (0.1358 * weight) + (0.00189 * hr * weight)
    else:
        return -5.92 + (0.0577 * hr) - (0.01677 * weight) + (0.00052 * hr * weight)
def TEE_dugas(hr, weight, age, male=True):
    if male:
        return (-16.1 + (hr[1:]*0.194) + (0.311*hr[:-1]) +
                (-0.002 * hr[1:]*hr[:-1]) + (-0.597*weight) +
                (0.353*age) + (0.007*hr[1:]*weight)) / 4.184
    else:
        return (-20.2 + (hr[1:]*0.397) + (0.155*hr[:-1]) +
                (-0.001 * hr[1:]*hr[:-1]) + (-0.174*weight) +
                (-0.08*age) + (0.001*hr[1:]*weight)) / 4.184

For these formulas to report reasonable data, we need to set a minimum floor of the calculated BMR that corresponds to the basal metabolic rate (BMR). Otherwise, we will report non-physically low total energy expenditure that is below the BMR.

In [ ]:
def calculate_bmr(weight, height, age, male=True):
    """
    Inputs:
        weight, in kg
        height, in cm
    Output: basal metabolic rate in kCal / minute. 
    
    Note that the original formula is kCal / day
    """
    height_m = height / 100  # Convert to meters.
    if male:
        if 10 <= age < 18:   bmr_day = 15.6 * weight + 266 * height_m + 299
        elif age < 30:       bmr_day = 14.4 * weight + 313 * height_m + 113
        elif age < 60:       bmr_day = 11.4 * weight + 313 * height_m - 137
        else: raise ValueError('Age is out of acceptable range')
    else:
        if 10 <= age < 18:   bmr_day = 9.40 * weight + 249 * height_m + 462
        elif age < 30:       bmr_day = 10.4 * weight + 615 * height_m - 282
        elif age < 60:       bmr_day = 8.18 * weight + 502 * height_m - 11.6
        else: raise ValueError('Age is out of acceptable range')
            
    return bmr_day / (24 * 60)
            

def adjust_calculated_tee(tee, bmr):
    """
    Replace the non-physically low TEE values with the BMR
    """
    output = np.copy(tee)
    bmr_flag = tee < bmr
    output[bmr_flag] = bmr
    return output

Occasionally during measurement, the actigraph whill not record HR data. If the gap is shorter than 15 minutes, we will interpolate the HR data. Otherwise, we will ignore this data.

In [ ]:
def remove_morning_evening_HR(df):
    """
    Remove the leading and trailing HR=0 datapoints
    """
    idx = np.arange(len(df))
    idx_start = idx[df.HR > 0].min()
    idx_end = idx[df.HR > 0].max()
    
    
#     plt.plot(df.Time_int, df.HR)
#     plt.plot(df.Time_int.iloc[idx_start:idx_end], 
#              df.HR.iloc[idx_start:idx_end], c='r')
#     plt.show()
    return df.iloc[idx_start:idx_end].copy()
    


def extend_hr_data_to_24_hours(df):
    """
    Ensure that the df contains 24 hours of data for the day. For these locations, we will initialize HR = -2
    """
    times_to_be_added = set(range(0, 24*60*60, 10)) - set(df.Time_int.values)
    times_to_be_added = list(times_to_be_added)
    times_to_be_added.sort()
    
    # Extend the df appropriately:
    df_extended = pd.DataFrame(
        data = np.zeros((len(times_to_be_added), len(df.columns))),
        columns = df.columns
    )
    df_extended['Time_int'] = times_to_be_added
    df_extended['Date'] = df.Date.values[0]
    df_extended['HR_corrected'] = -2 * np.ones(len(times_to_be_added))
    
    df = df.append(df_extended)
#     df = df.sort_values('Time_int')
    return df
    

def interpolate_hr(df):
    """
    For short gaps in the data, we will linearly interpolate the HR between the "start" and "end" of the gap.
    
    Data is saved in a new column, called 'HR_corrected'. During long gaps, the HR_corrected will remain zero.
    """
    df_output = df.copy()
    
    # Interpolate the non-zero HR data.
    idx_good_HR = df_output.HR > 0  # Data to construct the interpolation function
    hr_interpolate = interpolate.interp1d(
        df_output.Time_int[idx_good_HR], df_output.HR[idx_good_HR],
        bounds_error=False,
        fill_value=tuple(df_output.HR[idx_good_HR].values[(0, -1), ])
        )
    
    # Initialize and update the corrected HR data
    df_output['HR_corrected'] = df_output.HR  
    df_output.loc[df_output.HR == -1, 'HR_corrected'] = hr_interpolate(df_output.Time_int[df_output.HR == -1].values)
#     print(np.sum(df.HR == -1), np.sum(df.HR_corrected == -1))

    return df_output


def sanitize_hr_data(df):
    """
    We need to pre-process the HR data in a specific way.
    
    Leave HR = 0 data based on three criteria:
    1) HR = 0 at start of collection
    2) HR = 0 at end of collection
    3) There's more than a 15 minute gap between HR measurements
       
    Interpolate the HR data if the gap between "missing values" is short (< 15 minutes)
    """
    # Remove the HR=0 data at the start and end of the day:
    df = remove_morning_evening_HR(df)
    
    ### Find small gaps in the data, and flag as "HR = -1". We will then interplate these values.
    # First, calculate the "gap" between HR > 0 values.
    idx_search = df.index[df.HR > 0].values
    time_delta = df.Time_int[df.HR > 0].values[1:] - df.Time_int[df.HR > 0].values[:-1]
    
    # update the HR values appropriately.
    for i in np.argwhere(time_delta <= 15 * 60):  # find "short" gaps less than 15 minutes
        idx_start = idx_search[i][0] + 1
        idx_end = idx_search[i+1][0] - 1
        df.loc[idx_start: idx_end, 'HR'] = -1


    # Interpolate the HR data
    df = interpolate_hr(df)
        
    # Ensure we have 24 hours of data:
    df = extend_hr_data_to_24_hours(df)

    return df

Lastly, define a bunch of utility functions that will help us process the data.

In [ ]:
def convert_time(t):
    """
    Convert a timestamp into an integer, based on number of seconds elapsed since midnight
    """
    time = datetime.time.fromisoformat(str(t))
    timeint = time.second + 60 * time.minute + 60**2 * time.hour
    return timeint

def convert_date(d):
    """
    Convert a timestamp to a "datetime" data structure
    """
    date = datetime.datetime.fromisoformat(str(d))
    return date

def patient_id_from_file(file):
    idx = len('NOLS FA 18 HR Data_')
    return int(file[idx:idx+2])

# Begin processing the data

In [ ]:
src_dir = 'NOLS FA18 HR Data'
dest_dir = 'Processed_HR_data'
data_files = [f for f in os.listdir(src_dir) if f.startswith('NOLS FA 18 HR Data') and f.endswith('.xlsx')]
biodata_file = 'HR weight height age sex gender.xlsx'
biodata_df = pd.read_excel(biodata_file)

# Iterate over files:
for file in data_files:
    # Identify the patient ID:
    patient_ID = patient_id_from_file(file)
    if patient_ID not in biodata_df.id.values:
        print('Skip: {}... No Biodata in "{}"'.format(file, biodata_file))
        continue
    
    # Load the data:
    df = pd.read_excel(
        os.path.join(src_dir, file),
        skiprows=10
        ).rename(columns=lambda x: x.strip(' '))  # Strip whitespace from the headings

    # Convert the time to an integer... need to do this to let us manipulate the data!
    df['Time_int'] = df.Time.apply(np.vectorize(convert_time))

    # Which month is the data from?
    # If month <= 9... use "weight_kg_2" data (measurements from timepoint #2)
    # else... use "weight_kg_5" data (measurements from timepoint #5).
    month = convert_date(df.Date[0]).month

    # Look up the patient in the biodata_df to find metadata:
    if month <= 9:  # Use data from checkpoint #2
        patient_gender, patient_weight, patient_height, patient_age = biodata_df.loc[biodata_df.id == patient_ID,
                                                                     ('gender_2', 'weight_kg_2', 'height_cm_2', 'age_2')].values[0]
    elif month > 9: # use data from checkpoint #5
        patient_gender, patient_weight, patient_height, patient_age = biodata_df.loc[biodata_df.id == patient_ID,
                                                                     ('gender_5', 'weight_kg_5', 'height_cm_5', 'age_5')].values[0]
        
    patient_is_male = patient_gender == 'M'
    
    if np.isnan(patient_age):
        continue
    
    # Calculate the BMR:
    bmr = calculate_bmr(patient_weight, patient_height, patient_age, male=patient_is_male)

    # Assert that we don't have NaN data:
    if any(map(np.isnan, (patient_weight, patient_age))):
        print('Skip: {}'.format(file))
        print(patient_gender, patient_weight, patient_age)
        print()
        continue
                

    """
    Begin processing the data!
    1) Split the data file by date, creating a new DataFrame for each measurement
    2) For each date:
      + Filter to include only "waketime" HR data
      + Fix the HR data
      + Calculate the TEE
    3) Save into a new Excel spreadsheet, with each date as a worksheet
    """

    # Open the Excel file to accept output
    xlsx_writer = pd.ExcelWriter(
        os.path.join(dest_dir, 'processed_{}'.format(file)),
        engine='xlsxwriter')

    date_list = df.Date.unique()
    print(file)
    for date in date_list:
        try:
            # Filter by date, and pre-process the HR data:
            df_i = df[df.Date == date].copy()
            df_i = sanitize_hr_data(df_i)            
         
            if df_i.shape[0] > 1:
                print('\t{}'.format(date), end='... ')
            else:
                print('\tNO DATA: {}'.format(date))
                continue
        except:
#             print(df_i.shape)
            print('\tUNKNOWN ERROR: {}'.format(date))
            continue
            

        # Calculate the total energy expenditures, and add to the database:
        df_i['TEE_hiiloskorpy_light'] = TEE_hiiloskorpi_light(df_i.HR_corrected.values, patient_weight, male=patient_is_male) / 6
        df_i['TEE_hiiloskorpy_heavy'] = TEE_hiiloskorpi_heavy(df_i.HR_corrected.values, patient_weight, male=patient_is_male) / 6
        
        # Use Pandas DF slicing to select "light" vs "heavy" TEE values
        df_i['TEE_hiiloskorpy'] = df_i['TEE_hiiloskorpy_light'].values
        df_i.loc[df_i.HR_corrected > 90, 'TEE_hiiloskorpy'] = df_i.loc[df_i.HR_corrected > 90, 'TEE_hiiloskorpy_heavy']
        df_i['TEE_hiiloskorpy'] = adjust_calculated_tee(df_i['TEE_hiiloskorpy'].values, bmr / 6)

        # Sum the calculated TEE for the different methods:
        TEE_hiiloskorpy_sum = df_i['TEE_hiiloskorpy'].sum()
            
        # Calculate the waketime:
        participant_waketime = len(df_i) / 360  # Accounting for 10 second epochs --> 360 epochs / hour

        # Save the data to the output file:
        dt = pd.to_datetime(date)
        df_i.to_excel(xlsx_writer, sheet_name='{}-{}-{}_waketime'.format(
            dt.year, dt.month, dt.day))

        print('Waketime = {:02.1f} hrs'.format(participant_waketime))

    xlsx_writer.save()    
        